In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/Copy of A_05_Part_02_Dataset.zip"
with ZipFile(file_name, 'r',) as zip:
  zip.extractall()
  print('Done!!')

Done!!


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2
import glob

In [0]:
data_dir = 'A_05_Part_02_Dataset' #

In [5]:
#Define transforms for the training data and testing data
train_transforms = transforms.Compose([
                                       transforms.RandomRotation(30),
                                       transforms.Resize(224),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.252, 0.293, 0.288],
                                                            [0.146, 0.191, 0.193])])

test_transforms = transforms.Compose([
                                      transforms.Resize(224),
                                      transforms.ToTensor(),
                                       transforms.Normalize([0.252, 0.293, 0.288],
                                                            [0.146, 0.191, 0.193])])

valid_transforms = transforms.Compose([
                                      transforms.Resize(224),
                                      transforms.ToTensor(),
                                       transforms.Normalize([0.252, 0.293, 0.288],
                                                            [0.146, 0.191, 0.193])])

#pass transform here-in
train_data = datasets.ImageFolder(data_dir + '/Train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir + '/Validation', transform=valid_transforms)
# test_data = datasets.ImageFolder(data_dir + '/Test', transform=test_transforms)

#data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=12, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=12, shuffle=True)
# testloader = torch.utils.data.DataLoader(test_data, batch_size=12, shuffle=True)

print("Classes: ")
class_names = train_data.classes
print(class_names)

Classes: 
['covid-19', 'normal', 'pneumonia']


In [8]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16(pretrained=True)

vgg16.classifier[6].in_features = 1000
vgg16.classifier[6].out_features = 3

print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [0]:
#loss function and optimizer
Epochs = 10
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.0015, momentum=0.9)

In [16]:
#Training
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()
trainAccuracy =[]
validAccuracy = []
TrainLoss =[]
ValidLoss=[]
for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.item()))
        
    # torch.save(vgg16.state_dict(), 'vgg16_entire.pth')
    # print("Training stats \n")
    # tacc, tloss = getAccuracy(trainloader, "vgg")
    # trainAccuracy.append(tacc)
    # TrainLoss.append(tloss)
    # print("validation stats\n")
    # vacc , vloss = getAccuracy(validloader, "vgg")
    # validAccuracy.append(vacc)
    # ValidLoss.append(vloss)
    # print("\n>>tacc, tloss, vacc , vloss\n",tacc, tloss, vacc , vloss ,"\n............................\n")
print('Finished Training')







0it [00:00, ?it/s]

ValueError: ignored